In [ ]:
from SimilarityClustering import SimilarityClustering
import articles_data

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import cnouns as cn
from pymongo import MongoClient
from datetime import datetime

In [ ]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('clusters')

In [ ]:
catelist_path = '../datastore/category2.p'
# w2v_src_dir = "../datastore/w2v_src"
# w2v_path = "../datastore/sejongcorpus_w2v2.p"

w2v_src_dir = "../datastore/w2v_src2"
w2v_path = "../datastore/sejongcorpus_w2v2_2.p"

corpus_path = "../datastore/corpus2.p"
now = datetime.now()
prefix = int("%.2d%.2d"%(now.month, now.day))
prefix = 1104
prefix_str = "%d_00" % prefix

# Dict

In [1]:
from konlpy.tag import Mecab
import hanja
import re
import cnouns as cn
mecab = Mecab()

In [2]:
# train_df = articles_data.find_recent_articles(crawled_collection, catelist_path)

In [3]:
inp_str = u"최순실: 최순실 새누리당: 새누리당"

In [4]:
clean_text = cn.text_cleaning(inp_str)
pos_tags = mecab.pos(clean_text)
clean_text2 = cn.text_cleaning_without_special_ch(inp_str)
split_by_whitespace = clean_text2.split(' ')

In [5]:
for t in split_by_whitespace:
    print t

최순실
최순실
새누리당
새누리당


In [ ]:
morphs_ngrams_without_tag(pos_tags,3)

In [6]:
for t in cn.morphs_ngrams_without_tag(pos_tags,3):
    print t

최순
실
최순
실
새
누리
당
새
누리
당
최순실
최순실
실새
새누리
누리당
새누리
누리당
최순실새
실새누리
새누리당
새누리당


In [ ]:
n = 3
ngrams = zip(*[pos_tags[i:] for i in range(n)])
tt = [ngram for ngram in ngrams]
for t in tt:
    print t

In [ ]:
for t in pos_tags:
    print t[0], t[1]

# Doc2Vec

In [ ]:
size = articles_data.makeDataset(crawled_collection, w2v_src_dir, corpus_path)
print size

In [ ]:
articles_data.trainWord2Vec(w2v_src_dir, w2v_path, size)

# Model

In [ ]:
train_df = articles_data.find_recent_articles(crawled_collection, catelist_path)
sc = SimilarityClustering()
sc.train("cate", w2v_path, train_df, path="../datastore", prefix=prefix_str)

In [ ]:
sc = SimilarityClustering.load(path="../datastore", prefix=prefix_str)

# Save

In [ ]:
sc.save_to_db(prefix, clusters_collection)

# Test

In [ ]:
sc.print_clusters()

In [ ]:
sc.print_topics()

In [ ]:
sc.print_centers()

In [ ]:
clusters_collection.find_one({"cluster":1104032})